# Training Data

### File descriptions

    train.csv - the training set (contains 1458644 trip records)
    test.csv - the testing set (contains 625134 trip records)
    sample_submission.csv - a sample submission file in the correct format

### Data fields

    id - a unique identifier for each trip
    vendor_id - a code indicating the provider associated with the trip record
    pickup_datetime - date and time when the meter was engaged
    dropoff_datetime - date and time when the meter was disengaged
    passenger_count - the number of passengers in the vehicle (driver entered value)
    pickup_longitude - the longitude where the meter was engaged
    pickup_latitude - the latitude where the meter was engaged
    dropoff_longitude - the longitude where the meter was disengaged
    dropoff_latitude - the latitude where the meter was disengaged
    store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
    trip_duration - duration of the trip in seconds

In [1]:
%matplotlib inline
import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import syslog
import time

In [8]:
import dataloader
# import numpy as np
# import keras
# from keras.layers import *
# from keras.models import Model
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import syslog
DATA_PATH = 'datasets/train.csv'


def helloworld():
    print('hello world')


taxi_data = dataloader.load_data(DATA_PATH)

Load training data... usually takes 1 minute
Finished loading. 58.404911279678345 seconds.
longitude: -121.93334197998047 to -61.33552932739258 , latitude: 32.1811408996582 to 51.88108444213867


In [13]:
import dataloader
aj_3d, aj_4d = dataloader.create_am(taxi_data, -74.2, -73.7, 40.5, 40.9, 5, 4)
print(aj_4d.shape)

Generating adjacency matrice...


IndexError: index 20 is out of bounds for axis 0 with size 20

In [3]:
print('D/O', end='')
for x in range(21):
    print("%4d" % x, end=' ')

print(' ')
for i in range(0, 441, 21):
    x = i/21
    print("%2d" % x, end=' ')
    for x in aj[4][6][i:i+21].astype(int):
        print("%4d" % x, end=' ')
    print(' ')

D/O   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20  
 0    0    0    0    0    0    0    0    0    0    0    0    0    1    0    0    0    0    0    0    0    0  
 1    0    2    0    0    0    0    0    0    0    0    0    3    0    0    0    0    0    0    0    0    0  
 2    0    0    0    0    0    0    0    0    0    2    3    0    0    0    0    0    0    0    0    0    0  
 3    0    0    0    0    0    0    2    0    0    0    0    0    0    0    0    0    0    0    0    0    0  
 4    0    0    0    0    0    0    0    0    2    0    0    0    0    0    0    0    0    0    0    0    0  
 5    0    0    0    0    0    0    3   21    0    0    0    0    0    0    0    0    0    0    0    0    0  
 6   16    5    0    0    0    5   25    0    0    0    0    0    0    0    0    0    0    0    0    0    5  
 7   47    2   25    0   67  204   26    0    0    0    5    0    0    0    0    0    0    0    0    0    0  
 8    0   

In [6]:
####################REMOVE########################

#longitude range: -61 to -122, latitude range: 52-32, time range: 2016-01-01 to 2016-07-01
#print(len(trip))
#print(row.keys())
x_block = 5
y_block = 2
t_block = 183
n_block = x_block*y_block
n_input = n_block*n_block
logi_len = (maxlogi-minlogi)/x_block
lati_len = (maxlati-minlati)/y_block
mintime = datetime.strptime('2016-1-1 0:0:0', "%Y-%m-%d %H:%M:%S")

def block(logi, lati):
    x = int((logi-minlogi)/logi_len)
    y = int((lati-minlati)/lati_len)
    if x==x_block: x = x-1
    if y==y_block: y = y-1

    return y*x_block+x

#adjacency matrice, 183*24*100
#date from 0101 to 0107, time 0-23 stands for every hours
#there are 10 blocks in the map, and each people traveling is counted in 10*10=100 array
aj = np.zeros((t_block, 24, n_input))
for t in trip:
    orig = block(t[5], t[6])
    dest = block(t[7], t[8])
    date = (t[2]-mintime).days
    time = t[2].hour
    aj[date][time][orig+dest*n_block] += t[4]

NameError: name 'maxlogi' is not defined

## create adjacency matrice

In [4]:
start_time = time.time()

#longitude range: -61 to -122, latitude range: 52-32, time range: 2016-01-01 to 2016-07-01
#print(len(trip))
#print(row.keys())

w_block = 26
mintime = datetime.strptime('2016-1-1 0:0:0', "%Y-%m-%d %H:%M:%S")
logi_len = (maxlogi-minlogi)/5; lati_len = (maxlati-minlati)/2
minlogi = minlogi+3*logi_len; maxlogi = minlogi+logi_len
maxlati = minlati+lati_len

logi_len = logi_len/3; lati_len = lati_len/3
x_block = 3; y_block = 3
n_block = x_block*y_block
n_input = 100

# return 0-8 and 9
def block(logi, lati):
    if (logi>minlogi) and (logi<maxlogi) and (lati>minlati) and (lati<maxlati):
        x = int((logi-minlogi)/logi_len)
        y = int((lati-minlati)/lati_len)
        return y*x_block+x
    else:
        return 9

#adjacency matrice, 26*7*100
#26 weeks, 7 days
#there are 10 blocks in the map, and each people traveling is counted in 10*10=100 array
aj = np.zeros((26, 7, n_input))
for t in trip:
    orig = block(t[5], t[6])
    dest = block(t[7], t[8])
    date = (t[2]-mintime).days
    week, weekday= divmod(date, 7)
    if week<26:
        aj[week][weekday][orig+dest*n_block] += t[4]

elapsed_time = time.time() - start_time
print('generate adjacency matrice.', elapsed_time, 'seconds.')

generate adjacency matrice. 4.148188829421997 seconds.


## LSTM attempt

In [6]:
start_time = time.time()

# 26*7*100 to 26*20
inpE = Input((7, n_input))
outE = LSTM(units = 20, return_sequences=False, activation='tanh')(inpE)
encoder = Model(inpE,outE)

# 26*5 to 26*700 to 26*7*100
inpD = Input((20,))
outD = Reshape((20,1))(inpD)
outD = LSTM(7*n_input,return_sequences=False, activation='tanh')(outD)
outD = Reshape((7,n_input))(outD)
autoencoder = Model(inpD,outD)

autoencoder.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

autoencoder.fit(epochs=50,steps_per_epoch=10,batch_size=256)

elapsed_time = time.time() - start_time
start_time = time.time()
print('train model.', elapsed_time, 'seconds.')

train model. 0.9111177921295166 seconds.


In [ ]:
start_time = time.time()

#encode
encoderPredictions = encoder.predict(aj)
#decode
decoderPredictions = decoder.predict(encoderPredictions)

elapsed_time = time.time() - start_time
start_time = time.time()
print('predict.', elapsed_time, 'seconds.')

In [ ]:
print(aj.shape, encoderPredictions.shape, decoderPredictions.shape)

## PCA attempt

In [ ]:
start_time = time.time()

aj_re = aj.reshape(-1, 700)

pca = PCA(.95)
pca_aj = pca.fit_transform(aj_re)

elapsed_time = time.time() - start_time
start_time = time.time()
print('pca on adjacency matrice.', elapsed_time, 'seconds.')

In [ ]:
start_time = time.time()

pcainv_aj = pca.inverse_transform(pca_aj)
pcainv_aj = pcainv_aj.reshape(-1, 7, 100)
print(pcainv_aj.shape)

elapsed_time = time.time() - start_time
start_time = time.time()
print('pca inverse on adjacency matrice.', elapsed_time, 'seconds.')

## reference

Taxi data: https://www.kaggle.com/c/nyc-taxi-trip-duration/data

Using LSTM: https://blog.keras.io/building-autoencoders-in-keras.html

https://stackoverflow.com/questions/44647258/lstm-autoencoder

resize
normalize
reconstruction error
hidden layer 10
visualize